# *** recreate camb18_dataset ***

In [ ]:
## install
#!pip install camb

# import libraries

In [1]:
## import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## drive
#from google.colab import drive
#drive.mount('/content/drive/')

# directory

In [2]:
## directory
directory = 'camb18_dataset/'

# split up full-dataset as flow and fhigh 

# create flow and fhigh dataset

In [3]:
## split up TT_full as TT_flow and TT_fhigh
TT_full_fname = 'TT_full.csv'
TT_full       = pd.read_csv(directory + TT_full_fname)

ns_As   = TT_full.iloc[:,0:2]

TT_flow  = TT_full.iloc[:,2:30]
TT_flow  = pd.concat([ns_As, TT_flow], axis=1)
TT_flow.to_csv(directory + 'TT_flow.csv', index=False)

TT_fhigh = TT_full.iloc[:,30:]
TT_fhigh = pd.concat([ns_As, TT_fhigh], axis=1)
TT_fhigh.to_csv(directory + 'TT_fhigh.csv', index=False)

In [4]:
## split up TE_full as TE_flow and TE_fhigh
TE_full_fname = 'TE_full.csv'
TE_full       = pd.read_csv(directory + TE_full_fname)

ns_As   = TE_full.iloc[:,0:2]

TE_flow  = TE_full.iloc[:,2:30]
TE_flow  = pd.concat([ns_As, TE_flow], axis=1)
TE_flow.to_csv(directory + 'TE_flow.csv', index=False)

TE_fhigh = TE_full.iloc[:,30:].copy()
TE_fhigh = pd.concat([ns_As, TE_fhigh], axis=1)
TE_fhigh.to_csv(directory + 'TE_fhigh.csv', index=False)

In [5]:
## split up EE_full as EE_flow and EE_fhigh
EE_full_fname = 'EE_full.csv'
EE_full       = pd.read_csv(directory + EE_full_fname)

ns_As   = EE_full.iloc[:,0:2]

EE_flow  = EE_full.iloc[:,2:30]
EE_flow  = pd.concat([ns_As, EE_flow], axis=1)
EE_flow.to_csv(directory + 'EE_flow.csv', index=False)

EE_fhigh = EE_full.iloc[:,30:].copy()
EE_fhigh = pd.concat([ns_As, EE_fhigh], axis=1)
EE_fhigh.to_csv(directory + 'EE_fhigh.csv', index=False)

# create the binned_high-$\ell$ dataset

In [6]:
TT_fhigh_fname = 'TT_fhigh.csv'
TT_fhigh       = pd.read_csv(directory + TT_fhigh_fname)
TE_fhigh_fname = 'TE_fhigh.csv'
TE_fhigh       = pd.read_csv(directory + TE_fhigh_fname)
EE_fhigh_fname = 'EE_fhigh.csv'
EE_fhigh       = pd.read_csv(directory + EE_fhigh_fname)

In [7]:
df_cp = TT_fhigh[['ns','As']]

In [8]:
df_TT_fhigh = TT_fhigh.iloc[:,2:]
df_TE_fhigh = TE_fhigh.iloc[:,2:]
df_EE_fhigh = EE_fhigh.iloc[:,2:]

In [9]:
TT_fhigh = []
TE_fhigh = []
EE_fhigh = []

for i in range (len(df_TT_fhigh)):
    TT_fhigh.append(df_TT_fhigh.iloc[i,:].values)

for i in range (len(df_TE_fhigh)):
    TE_fhigh.append(df_TE_fhigh.iloc[i,:].values)

for i in range (len(df_EE_fhigh)):
    EE_fhigh.append(df_EE_fhigh.iloc[i,:].values)

In [10]:
## get the binned fhigh dataset for camb15
def binned(array, deltal, weights=1):
    binned_array = []
    n_bin        = int(len(array)/deltal) + 1  
    for i in range(n_bin):
        binn     = array[deltal*i:deltal*(i+1)]
        average  = np.average(binn)
        binned_array.append(average)
    return binned_array

In [11]:
hl_TT = np.arange(30, 2509)
hl_TE = np.arange(30, 1997)
hl_EE = np.arange(30, 1997)

In [12]:
deltal = 30

In [13]:
Dl_TE = []
Dl_EE = []
Dl_TT = []

for i in range(len(TT_fhigh)):
    Dl_TT.append(binned(TT_fhigh[i], deltal))
TT_fhbinned = np.array(Dl_TT)


for i in range(len(TE_fhigh)):
    Dl_TE.append(binned(TE_fhigh[i], deltal))   
TE_fhbinned = np.array(Dl_TE) 


for i in range(len(EE_fhigh)):
    Dl_EE.append(binned(EE_fhigh[i], deltal))
EE_fhbinned = np.array(Dl_EE) 

In [14]:
hlb_TT = []
hlb_TE = []
hlb_EE = []

for i in range(len(hl_TT)):
    hlb_TT.append(binned(hl_TT, deltal))
hlbinned_TT = np.array(hlb_TT).T

for i in range(len(hl_TE)):
    hlb_TE.append(binned(hl_TE, deltal))
hlbinned_TE = np.array(hlb_TE).T

for i in range(len(hl_EE)):
    hlb_EE.append(binned(hl_EE, deltal))
hlbinned_EE = np.array(hlb_EE).T

In [15]:
df_TT = pd.DataFrame(TT_fhbinned, columns=list(map(str, hlbinned_TT[:,0])))
df_TT = pd.concat([df_cp, df_TT],axis=1)

df_TE = pd.DataFrame(TE_fhbinned, columns=list(map(str, hlbinned_TE[:,0])))
df_TE = pd.concat([df_cp, df_TE],axis=1)

df_EE = pd.DataFrame(EE_fhbinned, columns=list(map(str, hlbinned_EE[:,0])))
df_EE = pd.concat([df_cp, df_EE],axis=1)

In [16]:
df_TT.to_csv(directory + 'TT_fhigh_binned.csv', index=False)
df_TE.to_csv(directory + 'TE_fhigh_binned.csv', index=False)
df_EE.to_csv(directory + 'EE_fhigh_binned.csv', index=False)

# create the binned_full dataset

In [17]:
TT_fl_fname = 'TT_flow.csv'
TT_fl       = pd.read_csv(directory + TT_fl_fname)
TE_fl_fname = 'TE_flow.csv'
TE_fl       = pd.read_csv(directory + TE_fl_fname)
EE_fl_fname = 'EE_flow.csv'
EE_fl       = pd.read_csv(directory + EE_fl_fname)

In [18]:
TT_bfh_fname = 'TT_fhigh_binned.csv'
TT_bfh       =  pd.read_csv(directory + TT_bfh_fname).drop(['ns','As'], axis=1)
TE_bfh_fname = 'TE_fhigh_binned.csv'
TE_bfh       = pd.read_csv(directory + TE_bfh_fname).drop(['ns','As'], axis=1)
EE_bfh_fname = 'EE_fhigh_binned.csv'
EE_bfh       = pd.read_csv(directory + EE_bfh_fname).drop(['ns','As'], axis=1)

In [19]:
TT_ffull = pd.DataFrame()
TE_ffull = pd.DataFrame()
EE_ffull = pd.DataFrame()

TT_ffull = pd.concat([TT_fl, TT_bfh], axis=1, sort='True')
TE_ffull = pd.concat([TE_fl, TE_bfh], axis=1, sort='True')
EE_ffull = pd.concat([EE_fl, EE_bfh], axis=1, sort='True')

In [20]:
TT_ffull.to_csv(directory + 'TT_full_binned.csv', index=False)
TE_ffull.to_csv(directory + 'TE_full_binned.csv', index=False)
EE_ffull.to_csv(directory + 'EE_full_binned.csv', index=False)

# create the dataset combinations

In [21]:
TT_bfh_fname = 'TT_fhigh_binned.csv'
TT_bfh       =  pd.read_csv(directory + TT_bfh_fname)
TE_bfh_fname = 'TE_fhigh_binned.csv'
TE_bfh       = pd.read_csv(directory + TE_bfh_fname).drop(['ns','As'], axis=1)
EE_bfh_fname = 'EE_fhigh_binned.csv'
EE_bfh       = pd.read_csv(directory + EE_bfh_fname).drop(['ns','As'], axis=1)

TE_ffull_fname = 'TE_full_binned.csv'
TE_ffull = pd.read_csv(directory + TE_ffull_fname).drop(['ns','As'], axis=1)
EE_ffull_fname = 'EE_full_binned.csv'
EE_ffull = pd.read_csv(directory + EE_ffull_fname).drop(['ns','As'], axis=1)

TE_fl_fname = 'TE_flow.csv'
TE_fl       = pd.read_csv(directory + TE_fl_fname).drop(['ns','As'], axis=1)
EE_fl_fname = 'EE_flow.csv'
EE_fl       = pd.read_csv(directory + EE_fl_fname).drop(['ns','As'], axis=1)

In [22]:
TTTEEE_bfh   = pd.DataFrame()
TTTEEE_bfhf  = pd.DataFrame()
TTTEEE_bfhfl = pd.DataFrame()

TTTEEE_bfh   = pd.concat([TT_bfh, TE_bfh, EE_bfh],     axis=1, sort='True')
TTTEEE_bfhf  = pd.concat([TT_bfh, TE_ffull, EE_ffull], axis=1, sort='True')
TTTEEE_bfhfl = pd.concat([TT_bfh, TE_fl, EE_fl],       axis=1, sort='True')

In [23]:
TTTEEE_bfh.to_csv(directory   + 'TT_fhigh_binned+TE_fhigh_binned+EE_fhigh_binned.csv', index=False)
TTTEEE_bfhf.to_csv(directory  + 'TT_fhigh_binned+TE_full_binned+EE_full_binned.csv',   index=False)
TTTEEE_bfhfl.to_csv(directory + 'TT_fhigh_binned+TE_flow+EE_flow.csv',                 index=False)

In [24]:
TE_bfh_fname = 'TE_fhigh_binned.csv'
TE_bfh       = pd.read_csv(directory + TE_bfh_fname).drop(['ns','As'], axis=1)
EE_bfh_fname = 'EE_fhigh_binned.csv'
EE_bfh       = pd.read_csv(directory + EE_bfh_fname).drop(['ns','As'], axis=1)

TE_ffull_fname = 'TE_full_binned.csv'
TE_ffull = pd.read_csv(directory + TE_ffull_fname)
TE_ffull_fname = 'TE_full_binned.csv'
TE_ffull = pd.read_csv(directory + TE_ffull_fname).drop(['ns','As'], axis=1)
EE_ffull_fname = 'EE_full_binned.csv'
EE_ffull = pd.read_csv(directory + EE_ffull_fname).drop(['ns','As'], axis=1)

TE_fl_fname = 'TE_flow.csv'
TE_fl       = pd.read_csv(directory + TE_fl_fname).drop(['ns','As'], axis=1)
EE_fl_fname = 'EE_flow.csv'
EE_fl       = pd.read_csv(directory + EE_fl_fname).drop(['ns','As'], axis=1)

In [25]:
TTTEEE_ffull  = pd.DataFrame()
TTTEEE_ffullh = pd.DataFrame()
TTTEEE_ffulll = pd.DataFrame()

TTTEEE_ffull  = pd.concat([TT_ffull, TE_ffull, EE_ffull], axis=1, sort='True')
TTTEEE_ffullh = pd.concat([TT_ffull, TE_bfh, EE_bfh],     axis=1, sort='True')
TTTEEE_ffulll = pd.concat([TT_ffull, TE_fl, EE_fl],       axis=1, sort='True')

In [26]:
TTTEEE_ffull.to_csv(directory  + 'TT_full_binned+TE_full_binned+EE_full_binned.csv',   index=False)
TTTEEE_ffullh.to_csv(directory + 'TT_full_binned+TE_fhigh_binned+EE_fhigh_binned.csv', index=False)
TTTEEE_ffulll.to_csv(directory + 'TT_full_binned+TE_flow+EE_flow.csv',                 index=False)